In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# get hco data
hco_data_istanbul = pd.read_csv('./data/hco_data_istanbul.csv')
hco_data_istanbul.info()


# get sales data
df = pd.read_csv('./data/istanbul_data.csv')
drop_columns = ['Year_Month', 'Lat', 'Lon', 'City']
df = df.drop(columns=drop_columns)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17653 entries, 0 to 17652
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   HcoID          17653 non-null  int64 
 1   HcoType        17653 non-null  object
 2   City           17653 non-null  object
 3   District       16410 non-null  object
 4   SalesRegionID  17653 non-null  int64 
 5   Lat            4576 non-null   object
 6   Lon            4576 non-null   object
 7   Status         17653 non-null  object
dtypes: int64(2), object(6)
memory usage: 1.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4201059 entries, 0 to 4201058
Data columns (total 16 columns):
 #   Column           Dtype  
---  ------           -----  
 0   MainDistID       int64  
 1   DistID           int64  
 2   Date             object 
 3   HcoID            int64  
 4   SalesRegionID_x  int64  
 5   BrandID          int64  
 6   SkuID            int64  
 7   IsReturn         int64  

# Data Preprocessing

In [7]:
# if IsReturn == 1, then make TotalQuantity negative
df['TotalQuantity'] = np.where(df['IsReturn'] == 1, -df['TotalQuantity'], df['TotalQuantity'])


In [8]:
data = df.copy()

data['Date'] = pd.to_datetime(data['Date'])
data['Week'] = data['Date'].dt.isocalendar().week
data['Month'] = data['Date'].dt.month
data['Quarter'] = data['Date'].dt.quarter
data['Year'] = data['Date'].dt.year

# if sales region x and y are the same save 
data = data[data['SalesRegionID_x'] == data['SalesRegionID_y']]

# merge region cols 
data['SalesRegionID'] = data['SalesRegionID_x'].combine_first(data['SalesRegionID_y'])
data.drop(columns=['SalesRegionID_x', 'SalesRegionID_y'], inplace=True)

data['Status'] = data['Status'].apply(lambda x: True if x == 'Active' else False)

data['District'] = data['District'].str.upper()
data['District'].unique()

array(['SANCAKTEPE', 'BAKIRKÖY', 'KÜÇÜKÇEKMECE', 'BAĞCILAR', 'BAYRAMPAŞA',
       'GAZİOSMANPAŞA', 'GAZIOSMANPAŞA', 'ZEYTINBURNU', 'FATİH',
       'ZEYTİNBURNU', 'EYÜP', 'BAHÇELİEVLER', 'GÜNGÖREN', 'SULTANGAZİ',
       'ESENLER', 'BÜYÜKÇEKMECE', 'BAHÇELIEVLER', 'SİLİVRİ', 'SULTANGAZI',
       nan, 'FATIH', 'ATAŞEHİR', 'AVCILAR', 'BAŞAKŞEHİR', 'ÜSKÜDAR',
       'ÜMRANİYE', 'BEYLİKDÜZÜ', 'PENDİK', 'TUZLA', 'KARTAL',
       'SULTANBEYLİ', 'ÇEKMEKÖY', 'PENDIK', 'SILIVRI', 'KADIKÖY', 'ŞIŞLI',
       'MALTEPE', 'ESENYURT', 'KAĞITHANE', 'BEYOĞLU', 'ŞİŞLİ', 'BEŞİKTAŞ',
       'BAŞAKŞEHIR', 'ÜMRANIYE', 'ATAŞEHIR', 'BEYKOZ', 'ŞİLE', 'GÖZTEPE',
       'ADALAR', 'ŞILE', 'ARNAVUTKÖY', 'SARIYER', 'BEŞIKTAŞ', 'ÇATALCA',
       'BEYLIKDÜZÜ', 'CEKMEKOY', 'EMİNÖNÜ', 'SULTANBEYLI', 'KINALIADA',
       'EYÜPSULTAN'], dtype=object)

In [9]:
name_cols = ['MainDistID', 'DistID', 'HcoID', 'SkuID']
categorical_cols = ['HcoType', 'SalesRegionID'] 
numerical_cols = ['PaidQty', 'FGQty', 'TotalQuantity', 'Week', 'Quarter', 'Month', 'Year']
boolean_cols = ['IsReturn', 'IsMrsOrder', 'Status']

# convert categorical columns to 'category' data type
for col in categorical_cols:
    data[col] = data[col].astype('category')

# convert numerical columns to 'int' data type
for col in numerical_cols:
    data[col] = data[col].astype('int')
    
# convert name columns to str data type
for col in name_cols:
    data[col] = data[col].astype(str)

# convert boolean columns to 'bool' data type
for col in boolean_cols:
    data[col] = data[col].astype(bool)
    
data.info()
raw_data = data.copy()

<class 'pandas.core.frame.DataFrame'>
Index: 4111852 entries, 126 to 4201058
Data columns (total 19 columns):
 #   Column         Dtype         
---  ------         -----         
 0   MainDistID     object        
 1   DistID         object        
 2   Date           datetime64[ns]
 3   HcoID          object        
 4   BrandID        int64         
 5   SkuID          object        
 6   IsReturn       bool          
 7   PaidQty        int64         
 8   FGQty          int64         
 9   IsMrsOrder     bool          
 10  HcoType        category      
 11  District       object        
 12  Status         bool          
 13  TotalQuantity  int64         
 14  Week           int64         
 15  Month          int64         
 16  Quarter        int64         
 17  Year           int64         
 18  SalesRegionID  category      
dtypes: bool(3), category(2), datetime64[ns](1), int64(8), object(5)
memory usage: 619.2+ MB


In [10]:
segment_1_sku_ids = ["568", "578", "1050"]
segment_2_sku_ids = ["319", "509", "615", "1018"]

all_sku_ids = segment_1_sku_ids + segment_2_sku_ids

data = data[data['SkuID'].isin(all_sku_ids)]

In [11]:
data.to_csv('data/istanbul_data_fully_preprocessed.csv', index=False)

In [17]:
# export it to data/istanbul_data_preprocessed.csv

df.to_csv('data/istanbul_data_preprocessed.csv', index=False)

df.set_index('Date', inplace=True)

In [ ]:
# Temporal Analysis
# Analyze sales trends over time
sales_trend = df.groupby('Date')['TotalQuantity'].sum()
sales_trend.plot(figsize=(10, 6))
plt.title('Total Sales Trend Over Time')
plt.xlabel('Date')
plt.ylabel('Total Quantity Sold')
plt.show()

In [ ]:
# Customer Segmentation
# Segment customers based on ordering behavior
customer_segmentation = df.groupby('HcoID')['TotalQuantity'].sum().reset_index()
customer_segmentation['Segment'] = pd.qcut(customer_segmentation['TotalQuantity'], q=4, labels=['Low', 'Medium', 'High', 'Very High'])
customer_segmentation['Segment'].value_counts()

In [ ]:
# Product Analysis
# Analyze sales volumes and trends for different SKUs
product_sales = df.groupby('SkuID')['TotalQuantity'].sum().sort_values(ascending=False)
product_sales.head(10).plot(kind='bar', figsize=(10, 6))
plt.title('Top 10 Best Selling SKUs')
plt.xlabel('SKU')
plt.ylabel('Total Quantity Sold')
plt.xticks(rotation=45)
plt.show()

# SKU	Total Quantity Sold	Average Monthly Sales	Peak Month Peak Quantity
product_analysis = df.groupby('SkuID').agg(TotalQuantity=('TotalQuantity', 'sum'),
                                           AverageMonthlySales=('TotalQuantity', 'mean'),
                                           PeakMonth=('Month', lambda x: x.value_counts().index[0]),
                                           PeakQuantity=('Month', lambda x: x.value_counts().iloc[0]))
product_analysis = product_analysis.sort_values('TotalQuantity', ascending=False)
product_analysis.head(10)
# print(product_analysis.head(10).to_latex())


In [ ]:
# Supply Chain Analysis
# Analyze inventory levels and turnover rates for top 10 SKUs
inventory_analysis = df.groupby('SkuID')['TotalQuantity'].agg(['sum', 'count']).rename(columns={'sum': 'TotalQuantity', 'count': 'TransactionCount'})
inventory_analysis['TurnoverRate'] = inventory_analysis['TotalQuantity'] / inventory_analysis['TransactionCount']
inventory_analysis = inventory_analysis.sort_values('TurnoverRate', ascending=False)
inventory_analysis.head(10)

# print(inventory_analysis.head(10).to_latex())

### Seçilenler

- I. Segment: 568 (challenge olan), 578, 1050
- II. Segment: 319 (en stabil olan), 509/615, 1018 (düşük satışlı olan)

In [ ]:
segment_1_sku_ids = ['568', '578', '1050']
segment_2_sku_ids = ['319', '509', '615', '1018']

In [ ]:
# SKU 568
sku_id = segment_1_sku_ids[0]
sku_sales = df[df['SkuID'] == sku_id].groupby('Date')['TotalQuantity'].sum()

sku_sales.plot(figsize=(10, 6))
plt.title(f'Sales Trend for SKU: {sku_id}')
plt.xlabel('Date')
plt.ylabel('Total Quantity Sold')
plt.show()

In [ ]:
# SKU 1050

sku_id = segment_1_sku_ids[2]
sku_sales = df[df['SkuID'] == sku_id].groupby('Date')['TotalQuantity'].sum()

sku_sales.plot(figsize=(10, 6))
plt.title(f'Sales Trend for SKU: {sku_id}')
plt.xlabel('Date')
plt.ylabel('Total Quantity Sold')
plt.show()

In [ ]:
# SKU 319

sku_id = segment_2_sku_ids[0]
sku_sales = df[df['SkuID'] == sku_id].groupby('Date')['TotalQuantity'].sum()

sku_sales.plot(figsize=(10, 6))
plt.title(f'Sales Trend for SKU: {sku_id}')
plt.xlabel('Date')
plt.ylabel('Total Quantity Sold')
plt.show()

In [ ]:
# Create a new dataframe for the selected SKUs
selected_skus = segment_1_sku_ids + segment_2_sku_ids
selected_skus_df = df[df['SkuID'].isin(selected_skus)]

# Create a pivot table for the selected SKUs
selected_skus_pivot = selected_skus_df.pivot_table(index='Date', columns='SkuID', values='TotalQuantity', aggfunc='sum')
selected_skus_pivot = selected_skus_pivot.fillna(0)

# Plot the sales trends for the selected SKUs
selected_skus_pivot.plot(figsize=(10, 6))
plt.title('Sales Trends for Selected SKUs')
plt.xlabel('Date')
plt.ylabel('Total Quantity Sold')
plt.show()

selected_skus_pivot.head()

In [ ]:
selected_skus_df.head()

In [ ]:
selected_skus_df.info()